# Downloading genbank accessions from .xlsx table

In [106]:
import pandas as pd
from Bio import Entrez
import os

In [107]:
spp_accession = pd.read_excel("spp_accession.xlsx", index_col=0)

In [108]:
# Changing NaN to None
spp_accession = spp_accession.where(pd.notnull(spp_accession), None)

In [109]:
spp_accession

,18S,28S,abdA,EF1,EF2,LWR,WNT
Species,,,,,,,
Formica_fusca,FJ824197.1,FJ824242.1,FJ824331.1,AY334415.1,EF013263.1,KJ920611.1,LT977286.1
Linepithema_humile,AH012894.2,AH012910.2,AY525327.1,EF013277.2,EF013277.2,FJ161830.1,AY233596.1
Pseudomyrmex_gracilis,AH012887.2,AH012903.2,DQ352945.1,None,None,KR829388.1,KR829015.1
Solenopsis_invicta,AY334566.1,AY325955.1,DQ352941.1,KP730149.1,EF013348.1,EF013642.1,KP730228.1


In [74]:
def create_dir(dir_name):
    os.makedirs(os.path.dirname(dir_name), exist_ok=True)

In [113]:
def download_acc_from_dataframe(dataframe):
    for species in dataframe.index:
        species = species.replace(" ", "_")
        for gene, accession in dataframe.loc[species].iteritems():
            gene_dir = "seed_fasta/{}/".format(gene)
            #create_dir(gene_dir)
            if accession:
                fasta_path = "seed_fasta/{}/{}-{}.fasta".format(gene,species, gene)
                create_dir(fasta_path)
                with open(fasta_path, "w") as fasta:
                    handle = Entrez.efetch(db='nucleotide', id=accession, rettype='fasta', retmode='text')
                    content = handle.read()
                    for line in content.split("\n"):
                        if line.startswith(">"):
                            line = ">{}-{} {}".format(species, gene, accession)
                        fasta.write("{}\n".format(line))
               # with open(fasta_path, "w") as fasta:
               #     handle = Entrez.efetch(db='nucleotide', id=accession, rettype='fasta', retmode='text')
               #     fasta.write(handle.read())
               # with open(fasta_path, "w")
               #     seqdata = SeqIO.read(fasta, "fasta")
               #     print(">{}-{}\n{}\n".format(species, gene, seqdata.seq))

In [110]:
def download_acc_from_dataframe(dataframe):
    for species in dataframe.index:
        species = species.replace(" ", "_")
        for gene, accession in dataframe.loc[species].iteritems():
            gene_dir = "seed_fasta/{}/".format(gene)
            #create_dir(gene_dir)
            if accession:
                fasta_path = "seed_fasta/{}/{}-{}.fasta".format(gene,species, gene)
                create_dir(fasta_path)
                with open(fasta_path, "w") as fasta:
                    handle = Entrez.efetch(db='nucleotide', id=accession, rettype='fasta', retmode='text')
                    content = handle.read()
                    for line in content.split("\n"):
                        if line.startswith(">"):
                            line = ">{}-{} {}".format(species, gene, accession)
                        fasta.write("{}\n".format(line))

In [111]:
download_acc_from_dataframe(spp_accession)

/home/gabriel/.local/lib/python3.8/site-packages/Bio/Entrez/__init__.py:619: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


In [112]:
# Testing how to extract species, gene and acession at once from dataframe
#for species in spp_accession.index:
#    for gene, accession in spp_accession.loc[species].iteritems():
#        print(species, gene, accession)

Formica_fusca 18S FJ824197.1
Formica_fusca 28S FJ824242.1
Formica_fusca abdA FJ824331.1
Formica_fusca EF1 AY334415.1
Formica_fusca EF2 EF013263.1
Formica_fusca LWR KJ920611.1
Formica_fusca WNT LT977286.1
Linepithema_humile 18S AH012894.2
Linepithema_humile 28S AH012910.2
Linepithema_humile abdA AY525327.1
Linepithema_humile EF1 EF013277.2
Linepithema_humile EF2 EF013277.2
Linepithema_humile LWR FJ161830.1
Linepithema_humile WNT AY233596.1
Pseudomyrmex_gracilis 18S AH012887.2
Pseudomyrmex_gracilis 28S AH012903.2
Pseudomyrmex_gracilis abdA DQ352945.1
Pseudomyrmex_gracilis EF1 None
Pseudomyrmex_gracilis EF2 None
Pseudomyrmex_gracilis LWR KR829388.1
Pseudomyrmex_gracilis WNT  KR829015.1
Solenopsis_invicta 18S AY334566.1
Solenopsis_invicta 28S AY325955.1
Solenopsis_invicta abdA DQ352941.1
Solenopsis_invicta EF1 KP730149.1
Solenopsis_invicta EF2 EF013348.1
Solenopsis_invicta LWR EF013642.1
Solenopsis_invicta WNT KP730228.1
